In [ ]:
import pandas as pd
import re
import string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np

## Data Preparation<a id='DataPreparation'></a>

This data set contains 6953 rows with 5 columns: Position, Company, Job Description, Review, and Location.

In [ ]:
# Load and view data
df = pd.read_csv('../input/data-scientist-job-market-in-the-us/alldata.csv')
df.head()

### Data Cleaning <a id='cleaning'></a>

In order to clean the data, I checked for the number of NaNs in each column. Since the Review column has the greatest number of NaNs, I decided to drop it all together since I won't be doing analysis on it. For the rest of the NaNs, I filtered the data frame to leave out all the rows containing null values. 

In [ ]:
# Check if there are any NaNs in the data
df.isnull().sum()

In [ ]:
# Drop column Review from the data
df.drop(columns = 'reviews', inplace = True)

In [ ]:
# Filtered the data set to remove the rest of the rows containing NaNs value
df.drop(index = df[df['position'].isnull()].index, inplace = True)
df.isnull().any()

### Populate New Columns <a id='newclumn'></a>

In order to aggregate the data, I've created City and State columns based on the Location column of the original data set. 

In [ ]:
# Create city and state columns to better aggregate the data
df['location'] = df.location.apply(lambda x: re.sub('\d*','',str(x)))
df['city'] = df.location.apply(lambda x: x.split(',')[0].strip())
df['state'] = df.location.apply(lambda x: x.split(',')[1].strip())
df['location'] = df['city']+ ', ' + df['state']
df.head()

## Exporatory Data Analysis<a id='eda'></a>

In this section, I'll utilize **pandas** and **matplotlib** to answer the following questions:

- What is the is the most common job to appear when searching for 'Data Science'?
- Which company hire the most Data Science job?
- From the data, which cities and state hire the most?

### Positions by Job Title <a id='title'></a>

Since position titles are varied from one company to another, the following code block will categorize the titles into 5 groups: Data Scientist, Machine Learning Engineer, Data Analyst, Data Science Manager, and Others.

*Note: I want to give credit to Pramod Manjegowda for the following code block. Pramad followed a machine learning approach toward this data set. You can take a look at his notebook <a href ='https://www.kaggle.com/pramod7/data-science-jobs-opening-in-us-analysis-ml'>here</a>.* 

In [ ]:
# Group position name into 5 types
data = df.copy()
data['position']=[x.upper() for x in data['position']]
data.loc[data.position.str.contains("SCIENTIST"), 'position'] = 'Data Scientist'

data.loc[data.position.str.contains('ENGINEER'),'position']='Machine Learning Engineer'
data.loc[data.position.str.contains('PRINCIPAL STATISTICAL PROGRAMMER'),'position']='Machine Learning Engineer'
data.loc[data.position.str.contains('PROGRAMMER'),'position']='Machine Learning Engineer'
data.loc[data.position.str.contains('DEVELOPER'),'position']='Machine Learning Engineer'

data.loc[data.position.str.contains('ANALYST'), 'position'] = 'Data Analyst'
data.loc[data.position.str.contains('STATISTICIAN'), 'position'] = 'Data Analyst'

data.loc[data.position.str.contains('MANAGER'),'position']='Data Science Manager'
data.loc[data.position.str.contains('CONSULTANT'),'position']='Data Science Manager'
data.loc[data.position.str.contains('DATA SCIENCE'),'position']='Data Science Manager'
data.loc[data.position.str.contains('DIRECTOR'),'position']='Data Science Manager'

data.position=data[(data.position == 'Data Scientist') | (data.position == 'Data Analyst') | (data.position == 'Machine Learning Engineer') | (data.position == 'Data Science Manager')]
data.position=['Others' if x is np.nan else x for x in data.position]

In [ ]:
title = data.groupby(['position']).count().sort_values('company')

title['company'].plot(kind='barh',figsize = (10,5))
plt.xlabel('Count', size = 12)
plt.ylabel('')
plt.yticks(size = 10)
plt.xticks(size = 10)
plt.title('Number of Positions by Job Title', size = 20)
plt.show()


### Positions by Companies <a id='company'></a>

In [ ]:
company = df.groupby(['company']).count().sort_values('position').tail(20)

company['position'].plot(kind='barh',figsize = (10,5))
plt.xlabel('Count', size = 12)
plt.ylabel('')
plt.yticks(size = 10)
plt.xticks(size = 10)
plt.title('Number of Positions by Companies (Top 20)', size = 20)
plt.show()

From the chart, we can see that Amazon.com hire the most candidate, follow by Ball Aerospace, Microsoft and Google. 

### Positions by Cities <a id='city'></a>

In [ ]:
city = df.groupby(['location']).count().sort_values('position').tail(20)

city['position'].plot(kind='barh',figsize = (10,5))
plt.xlabel('Count', size = 12)
plt.ylabel('')
plt.yticks(size = 10)
plt.xticks(size = 10)
plt.title('Number of Positions by Cities (Top 20)', size = 20)
plt.show()

It appears that the top 5 cities that hire the most data science related job are New York, Seattle, Cambridge, Boston, and San Francisco. It makes sense since those cities are the technology hub of the country. 

### Positions by States <a id='state'></a>

In [ ]:
state = df.groupby('state').count().sort_values('position',ascending = False)

state['position'].plot(kind = 'bar',figsize = (10,5) ,width = 0.85)
plt.xlabel('')
plt.ylabel('Count',size = 12)
plt.title('Number of Positions by State', size = 20)
plt.yticks(size = 10)
plt.xticks(size = 10, rotation = 720)
plt.show()

Even though the city with the highest number of positions is New York, the highest number of position by state is California, follows by Massachusetts and Washington. 

### Position by State and Job Title <a id='statetitle'></a>

In [ ]:
data = data[data['position'] != 'Others']
i = 1
color = ['#A92420','#8A6FDF','#135390','#FDA649']
fig = plt.figure(figsize=(20,10))
for position in data.position.unique():
    x = data[data['position']== str(position)].groupby(['state']).count().sort_values('company')
    plt.subplot(2, 2, i)
    i += 1
    plt.bar(x.index,x['company'], color = color[i-2])
    plt.xlabel('')
    plt.xticks(size = 10)
    plt.title(str(position), size = 15)
plt.show()

- For Data Science Manager, Data Scientist and Machine Learning Engineer, California is the state that hire the most position
- For Data Analyst, it seems that New York hires the most position, follow closely by California

## Text Analysis <a id='textanalysis'></a>

In this section, I will focus on the Job Description column of the data. By using libraries like **re, wordcloud, matplotlib**, I hope to gain further insights on the requirements for the field of data science. I will try to answer the following questions:
- What are the companies looking for when hiring?
- How many years of experience do they required?
- What level of education do the companies prefer?

In [ ]:
# Example of a description value
df.description.values[0][0:int(len(df.description.values[0])/2)]

### Text Cleaning and Prepration <a id='textclean'></a>

In [ ]:
# Combine the desciptions by the job tilte
data = data.groupby('position').agg(lambda col: ' '.join(col))
data = data[['description']]

In [ ]:
# Create a function to clean text data
def clean_text(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text).lower() #remove punctutations
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n',' ',text)
    return text

In [ ]:
# Clean the text data and remove the job title 'Others'
clean = lambda x :clean_text(x)
df_clean = pd.DataFrame(data.description.apply(clean))
df_clean = df_clean[df_clean.index != 'Others'].copy()

In [ ]:
# Lemmentize the text data to improve analysis
lemmer = WordNetLemmatizer()
df_clean['description'] = df_clean.description.apply(lambda x: word_tokenize(x))
df_clean['description'] = df_clean.description.apply(lambda x : [lemmer.lemmatize(y) for y in x])
df_clean['description'] = df_clean.description.apply(lambda x: ' '.join(x))

In [ ]:
# Add words that frequently appear in the descriptions but carry no value to the list of stop words
from sklearn.feature_extraction import text
extra_stopword = ['data','experience','work','team','will','skill','year','skills']
stop_words = text.ENGLISH_STOP_WORDS.union(extra_stopword)

### Word Cloud <a id='wordcloud'></a>

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(stopwords=stop_words, background_color="white", colormap="Dark2",
             random_state=42, collocations = False, width=1600, height=800)
i = 0
fig = plt.figure(figsize=(15,8))
for x in df_clean.description.index:
    wc.generate(df_clean.description[str(x)])
    
    i += 1
    fig.add_subplot(2, 2, i)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(str(x), size = 15)
plt.show()

Here are my interpretations from looking at the WordCloud:
- Data Analyst will be doing **research**, **analysis** and **provide** insights to facilitate better **business** decision.
- Data Science Manager will be in charge of **developing** **product** to help **business** serve its **customer** better.
- Data Scientist will be doing **research**, implementing **machine learning** and building **model** to come up with business solution.
- Machine Learning Engineer will **design** and **develop** **software** for business or customer. 

### Year of Experience Required <a id='experience'></a>

In the following section, I will utilize regular expression to search and locate text strings with in a document.

*Note: I'm still a novice with regex so any recommendation on how to improve my matches will be very much appreciated!*

In [ ]:
text = df.description.values

In [ ]:
# Print out the first 5 examples of matches
limit = 0
for t in text:
    for sentance in t.split('\n'):
        if 'experience' in sentance:
            year = re.findall("\d{1,2}\+? year", sentance)
            if len(year)==1:
                print(year[0])
                print(sentance)
                print("*"*20)
                limit +=1
    if limit >= 5:
        break

In [ ]:
# Compile the year value found into a list
experience_req = []
for t in text:
    for sentance in t.split('\n'):
        if 'experience' in sentance:
            year = re.findall("\d{1,2}\+? year", sentance)
            if len(year)==1:
                num = year[0].split(' ')
                experience_req.append(num[0])

In [ ]:
# Remove the '+' sign after year value
for n,i in enumerate(experience_req):
    if "+" in i:
        experience_req[n] = re.sub(r'\+','',i)
experience_req = [int(item) for item in experience_req]

In [ ]:
# Remove outliers
for n,i in enumerate(experience_req):
    if i >= 20:
        experience_req.pop(n)

In [ ]:
plt.figure(figsize = (10,5))
plt.hist(experience_req,bins = list(range(0,21,2)), align = 'left')
plt.title('Experience Required Distribution', size = 15)
plt.ylabel('Bin Count')
plt.xlabel('Year of Expereience', size = 12)
plt.show()
print(f'The average year of experience required is {round(np.mean(experience_req),2)} years')

### Skill Requirement <a id='skill'></a>

In [ ]:
# Create a regex search function
def count_text(patt,text):
    pattern = re.compile(patt)
    count = 0
    for t in text:
        if pattern.search(t):
            count+=1
    return count

In [ ]:
# Create a data frame with skills name and regex pattern to search with
skills = ['R','Python','Hadoop','SQL','Tableau','TensorFlow','Agile','Power BI','SSaS','Algorithm','Java','Visualization']

skill_patt = ['\WR\W+\s*','(?i)\WPython\W','(?i)\WHadoop\W?','(?i)SQL\w*','(?i)\WTableau\W?',
              "(?i)\WTensorFlow\W?","(?i)\WAgile\W?","(?i)\WPower\s?BI\W?",
             "(?i)\WSSAS\W?","(?i)\WAlgorithms?\W?",'(?i)Java\w*','(?i)\WVisualization\W?']

skill_df =pd.DataFrame(
    {"skill": skills,
     "regex_pattern":skill_patt})

In [ ]:
# Iterate through the list of skill using the search function created
i = []
for x in skill_df['regex_pattern']:
    i.append(count_text(x,text))
skill_df['count'] = i
skill_df['ptg'] = round(skill_df['count']/len(text),2)
skill_df

In [ ]:
x = skill_df.sort_values(by = 'ptg')
ax =x['ptg'].plot(kind = "barh",figsize = (10,5))
ax.set_title('Skills as Percentage of Total Job Description', size = 15)
ax.set_yticklabels(x['skill'], size = 12)
ax.set_xticklabels(['{:.0%}'.format(x) for x in ax.get_xticks()])
plt.show()

### Degree Requirement <a id='degree'></a>

In [ ]:
# Define regex pattern and seach for PhD
pattern = re.compile('(?i)\WPh.?D\W')
pattern2 = re.compile('(?i)\WDoctorate\W')
count = 0
for t in text:
    if pattern.search(t):
        count +=1
    elif pattern2.search(t):
        count +=1
degree = {"PhD": count}

In [ ]:
# Define regex pattern and seach for Master 
pattern = re.compile("(?i)\WMasters?'?s?\W")
pattern2 = re.compile('(?i)\WM.?S\W')
count = 0
for t in text:
    if pattern.search(t):
        count +=1
    elif pattern2.search(t):
        count +=1
degree.update({"Master":count})

In [ ]:
degree = pd.DataFrame.from_dict(degree,orient='index',
                       columns=[ 'count'])
degree['ptg'] = degree['count']/len(text)

In [ ]:
ax =degree['ptg'].plot(kind = "bar", figsize =(10,5))
ax.set_title('Percentage of Total Documents')
ax.set_xticklabels(degree.index)
ax.set_yticklabels(['{:.0%}'.format(x) for x in ax.get_yticks()])
plt.show()